# Neural Networks with PyTorch

In this assignment, we are going to train a Neural Networks on the Japanese MNIST dataset. It is composed of 70000 images of handwritten Hiragana characters. The target variables has 10 different classes.

Each image is of dimension 28 by 28. But we will flatten them to form a dataset composed of vectors of dimension (784, 1). The training process will be similar as for a structured dataset.

<img src='https://drive.google.com/uc?id=16TqEl9ESfXYbUpVafXD6h5UpJYGKfMxE' width="500" height="200">

Your goal is to run at least 3 experiments and get a model that can achieve 80% accuracy with not much overfitting on this dataset.

Some of the code have already been defined for you. You need only to add your code in the sections specified (marked with **TODO**). Some assert statements have been added to verify the expected outputs are correct. If it does throw an error, this means your implementation is behaving as expected.

Note: You can only use fully-connected and dropout layers for this assignment. You can not convolution layers for instance

# 1. Import Required Packages

[1.1] We are going to use numpy, matplotlib and google.colab packages

In [ ]:
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 2. Download Dataset

We will store the dataset into your personal Google Drive.


[2.1] Mount Google Drive

In [ ]:
drive.mount('/content/gdrive')

[2.2] Create a folder called `DL_ASG_1` on your Google Drive at the root level

In [ ]:
! mkdir -p /content/gdrive/MyDrive/DL_ASG_1

[2.3] Navigate to this folder

In [ ]:
%cd '/content/gdrive/MyDrive/DL_ASG_1'

[2.4] Show the list of item on the folder

In [ ]:
!ls

[2.4] Dowload the dataset files to your Google Drive if required

In [ ]:
import requests
from tqdm import tqdm
import os.path

def download_file(url):
    path = url.split('/')[-1]
    if os.path.isfile(path):
        print (f"{path} already exists")
    else:
      r = requests.get(url, stream=True)
      with open(path, 'wb') as f:
          total_length = int(r.headers.get('content-length'))
          print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))
          for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
              if chunk:
                  f.write(chunk)

url_list = [
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
    'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'
]

for url in url_list:
    download_file(url)

[2.5] List the content of the folder and confirm files have been dowloaded properly

In [ ]:
! ls

# 3. Load Data

[3.1] Import the required modules from PyTorch

In [ ]:
# TODO (Students need to fill this section)
import torch
import torchvision
import seaborn as sns
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import make_grid
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import pandas as pd

[3.2] **TODO** Create 2 variables called `img_height` and `img_width` that will both take the value 28

In [ ]:
# TODO (Students need to fill this section)
img_height = 28
img_width = 28

[3.3] Create a function that loads a .npz file using numpy and return the content of the `arr_0` key

In [ ]:
def load(f):
    return np.load(f)['arr_0']

[3.4] **TODO** Load the 4 files saved on your Google Drive into their respective variables: x_train, y_train, x_test and y_test

In [ ]:
# TODO (Students need to fill this section)
x_train = load("/content/gdrive/MyDrive/DL_ASG_1/kmnist-train-imgs.npz")
x_test = load("/content/gdrive/MyDrive/DL_ASG_1/kmnist-test-imgs.npz")
y_train = load("/content/gdrive/MyDrive/DL_ASG_1/kmnist-train-labels.npz")
y_test = load("/content/gdrive/MyDrive/DL_ASG_1/kmnist-test-labels.npz")

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

[3.5] **TODO** Using matplotlib display the first image from the train set and its target value

In [ ]:
# TODO (Students need to fill this section)
for i in range(6, 9):
    plt.subplot(330 + (i+1))
    plt.imshow(x_train[i].squeeze(), cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
## checking imbalances
from collections import Counter

y_train_counts = Counter(y_train)

sns.set(style="ticks", context="talk", font_scale=1)
plt.style.use("dark_background")
plt.figure(figsize=(20, 10))

# Plot the counts
ax = plt.bar(y_train_counts.keys(), y_train_counts.values(), color='grey')

# Add count labels on top of bars
for x, y in zip(ax.patches, y_train_counts.values()):
    plt.text(x.get_x() + x.get_width() / 2., y + 30, y, ha='center', size=30)

plt.xticks(rotation=0)
sns.despine()

plt.show()

There are no class imbalances in the data set, all the classes appear with the same frequency.

# 4. Prepare Data

[4.1] **TODO** Reshape the images from the training and testing set to have the channel dimension last. The dimensions should be: (row_number, height, width, channel)

In [ ]:
# TODO (Students need to fill this section)
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)


[4.2] **TODO** Cast `x_train` and `x_test` into `float32` decimals

In [ ]:
# TODO (Students need to fill this section)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')


[4.3] **TODO** Standardise the images of the training and testing sets. Originally each image contains pixels with value ranging from 0 to 255. after standardisation, the new value range should be from 0 to 1.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# TODO (Students need to fill this section)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, 28*28))
x_test = scaler.transform(x_test.reshape(-1, 28*28))


[4.4] **TODO** Create a variable called `num_classes` that will take the value 10 which corresponds to the number of classes for the target variable

In [ ]:
# TODO (Students need to fill this section)
num_classes = 10

[4.5] **TODO** Convert the target variable for the training and testing sets to a binary class matrix of dimension (rows, num_classes).

For example:
- class 0 will become [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
- class 1 will become [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
- class 5 will become [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
- class 9 will become [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [ ]:
# TODO (Students need to fill this section)
y_train_binary = np.eye(num_classes)[y_train]
y_test_binary = np.eye(num_classes)[y_test]

# 5. Define Neural Networks Architecure

[5.1] Set the seed in PyTorch for reproducing results



In [ ]:
# TODO (Students need to fill this section)
torch.manual_seed(0)

[5.2] **TODO** Define the architecture of your Neural Networks and save it into a variable called `model`

In [ ]:
# TODO (Students need to fill this section)
class ClassifierWithDropout(nn.Module):
    def __init__(self):
        super(ClassifierWithDropout, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, num_classes)


        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc2(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc3(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.fc4(x))
        x = self.dropout(x)
        x = nn.functional.log_softmax(self.fc5(x), dim=1)
        return x


num_classes = 10


model = ClassifierWithDropout()


[5.2] **TODO** Print the summary of your model

In [ ]:
# TODO (Students need to fill this section)
print(model)

# 6. Train Neural Networks

[6.1] **TODO** Create 2 variables called `batch_size` and `epochs` that will  respectively take the values 128 and 500

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs =  500

[6.2] **TODO** Compile your model with the appropriate loss function, the optimiser of your choice and the accuracy metric

In [ ]:
# TODO (Students need to fill this section)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


[6.3] **TODO** Train your model
using the number of epochs defined. Calculate the total loss and save it to a variable called total_loss.

In [ ]:
# TODO (Students need to fill this section)
import matplotlib.pyplot as plt
import numpy as np

train_losses = []
val_losses = []
accuracies = []

train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train_binary))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(torch.tensor(x_test), torch.tensor(y_test_binary))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.argmax(target, dim=1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(torch.argmax(target, dim=1)).sum().item()

    train_losses.append(total_loss / len(train_loader))
    accuracies.append(100. * correct / total)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, torch.argmax(target, dim=1)).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(torch.argmax(target, dim=1)).sum().item()

    val_losses.append(val_loss / len(val_loader))

    if epoch % 50 == 0:
        print(f"Epoch: {epoch+1}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}, Accuracy: {accuracies[-1]}%")

# Plotting
epochs_range = range(1, epochs+1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_losses, label='Train Loss', color='blue')
plt.plot(epochs_range, val_losses, label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, accuracies, label='Accuracy', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy')
plt.legend()

plt.show()


[6.4] **TODO** Test your model.  Initiate the model.eval() along with torch.no_grad() to turn off the gradients.


In [ ]:
# TODO (Students need to fill this section)

model.eval()
with torch.no_grad():
    test_output = model(torch.tensor(x_test))
    test_loss = criterion(test_output, torch.argmax(torch.tensor(y_test_binary), dim=1))

predicted = torch.argmax(test_output, 1)
accuracy = (predicted == torch.argmax(torch.tensor(y_test_binary), dim=1)).sum().item() / len(y_test)
print(f"Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}")


# 7. Analyse Results

[7.1] **TODO** Display the performance of your model on the training and testing sets

In [ ]:
# TODO (Students need to fill this section)
from sklearn.metrics import classification_report, confusion_matrix
model.eval()
with torch.no_grad():
    test_output = model(torch.tensor(x_test))
    test_loss = criterion(test_output, torch.argmax(torch.tensor(y_test_binary), dim=1))

predicted = torch.argmax(test_output, 1)

accuracy = (predicted == torch.argmax(torch.tensor(y_test_binary), dim=1)).sum().item() / len(y_test)
y_test_np = torch.argmax(torch.tensor(y_test_binary), dim=1).numpy()
predicted_np = predicted.numpy()

report = classification_report(y_test_np, predicted_np)

print(f"Test Loss: {test_loss.item()}")
print(f"Test Accuracy: {accuracy}")
print("Classification Report:")
print(report)



[7.2] **TODO** Plot the learning curve of your model

[7.3] **TODO** Display the confusion matrix on the testing set predictions

In [ ]:
# TODO (Students need to fill this section)
conf_matrix = confusion_matrix(y_test, predicted)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten the input images
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.log_softmax(self.fc5(x), dim=1)
        return x

model = Classifier()

In [ ]:
# TODO (Students need to fill this section)
print(model)

In [ ]:
# TODO (Students need to fill this section)
batch_size = 128
epochs =  500

In [ ]:
# TODO (Students need to fill this section)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# TODO (Students need to fill this section)
import matplotlib.pyplot as plt
import numpy as np

train_losses = []
val_losses = []
accuracies = []

train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train_binary))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(torch.tensor(x_test), torch.tensor(y_test_binary))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.argmax(target, dim=1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(torch.argmax(target, dim=1)).sum().item()

    train_losses.append(total_loss / len(train_loader))
    accuracies.append(100. * correct / total)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, torch.argmax(target, dim=1)).item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(torch.argmax(target, dim=1)).sum().item()

    val_losses.append(val_loss / len(val_loader))

    if epoch % 50 == 0:
        print(f"Epoch: {epoch+1}, Train Loss: {train_losses[-1]}, Validation Loss: {val_losses[-1]}, Accuracy: {accuracies[-1]}%")

# Plotting
epochs_range = range(1, epochs+1)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_losses, label='Train Loss', color='blue')
plt.plot(epochs_range, val_losses, label='Validation Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Train and Validation Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, accuracies, label='Accuracy', color='green')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy')
plt.legend()

plt.show()



In [ ]:
# TODO (Students need to fill this section)

from sklearn.metrics import classification_report, confusion_matrix
model.eval()
with torch.no_grad():
    test_output = model(torch.tensor(x_test))
    test_loss = criterion(test_output, torch.argmax(torch.tensor(y_test_binary), dim=1))

predicted = torch.argmax(test_output, 1)

accuracy = (predicted == torch.argmax(torch.tensor(y_test_binary), dim=1)).sum().item() / len(y_test)
y_test_np = torch.argmax(torch.tensor(y_test_binary), dim=1).numpy()
predicted_np = predicted.numpy()

report = classification_report(y_test_np, predicted_np)

print(f"Test Loss: {test_loss.item()}")
print(f"Test Accuracy: {accuracy}")
print("Classification Report:")
print(report)



In [ ]:
conf_matrix = confusion_matrix(y_test, predicted)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()